In [ ]:
import os
import pandas as pd
import numpy as np
import math
import statistics as stats

# Please complete the compute_apo function
# You are given a DataFrame named df containing the Adj Close price of a stock
# Use the provided APO formula to get APO for each day
# Add a new column named "APO" to df and return df["APO"]
def compute_apo(df):

    # calculate the 20-day exponential moving average (fast ema)
    # set min_periods = 20 in the ewm function
    df['ema_20'] = df[["Adj Close"]].ewm(span=20,min_periods = 20).mean()

    # calculate the 50-day exponential moving average (slow ema)
    # set min_periods = 50 in the ewm function
    df['ema_50'] = df[["Adj Close"]].ewm(span = 50,min_periods = 50).mean()

    # APO = fast ema - slow ema
    df['APO'] = df['ema_20'].subtract(df['ema_50'])
    
    return df

# Please complete the trade_with_risk_controls function
# Continue with the DataFrame df after you added a column for "APO"
# "APO" is just a technical indicator that will decide how we trade
# However, we now take risk management into consideration
def trade_with_risk_controls(df):
    # -- DO NOT MODIFY THE CODE BELOW -- # 
    # Variables for trade, position and P&L management
    last_buy_price = 0 # price of last buy trade
    last_sell_price = 0 # price of last sell trade
    position = 0 # current position
    buy_sum_price_quantity = 0 # sum of (buy_trade_price * buy_trade_quantity)
    buy_sum_quantity = 0 # sum of (buy_trade_quantity)
    sell_sum_price_quantity = 0 # sum of (sell_trade_price * sell_trade_quantity)
    sell_sum_quantity = 0 # sum of (sell_trade_quantity)
    unrealized_pnl = 0 # Mark-to-market P&L
    realized_pnl = 0 # Realized P&L
    
    # Constants that define strategy behavior/thresholds
    APO_VALUE_FOR_BUY_ENTRY = -10 # APO trading signal value below which to enter buy-orders/long-position
    APO_VALUE_FOR_SELL_ENTRY = 10 # APO trading signal value above which to enter sell-orders/short-position
    MIN_PRICE_MOVE_FROM_LAST_TRADE = 10 # Minimum price change since last trade before considering trading again, this is to prevent over-trading at/around same prices
    MIN_NUM_SHARES_PER_TRADE = 1
    MAX_NUM_SHARES_PER_TRADE = 50
    INCREMENT_NUM_SHARES_PER_TRADE = 2
    num_shares_per_trade = MIN_NUM_SHARES_PER_TRADE # Beginning number of shares to buy/sell on every trade
    num_shares_history = [] # history of num-shares
    abs_position_history = [] # history of absolute-position
    # Constants/variables that are used to compute standard deviation as a volatility measure
    SMA_NUM_PERIODS = 20 # look back period
    # Risk Controls and increments risk limits when we have good/bad months
    risk_limit_weekly_stop_loss = -100
    INCREMENT_RISK_LIMIT_WEEKLY_STOP_LOSS = -300
    risk_limit_monthly_stop_loss = -200
    INCREMENT_RISK_LIMIT_MONTHLY_STOP_LOSS = -600
    risk_limit_max_position = 5
    INCREMENT_RISK_LIMIT_MAX_POSITION = 5
    max_position_history = [] # history of max-position-size
    RISK_LIMIT_MAX_POSITION_HOLDING_TIME_DAYS = 30 * 5
    risk_limit_max_trade_size = 5
    INCREMENT_RISK_LIMIT_MAX_TRADE_SIZE = 2
    max_trade_size_history = [] # history of max-trade-size
    df["risk_violated"] = False
    traded_volume = 0
    current_pos = 0
    current_pos_start = 0
    last_risk_change_index = 50
    risk_violated = False
    # calculate rolling standard deviation and divided by 15 (common factor)
    df["stdev_factor"] = df["Adj Close"].rolling(20).std() / 15
    df["Order"] = 0
    df["Positions"] = 0
    df["PnL"] = 0
    df["risk_violated"] = False
    
    df["num_shares_history"] = MIN_NUM_SHARES_PER_TRADE
    df["abs_position_history"] = 0
    df["max_trade_size_history"] = risk_limit_max_trade_size
    df["max_position_history"] = risk_limit_max_position
    MIN_PROFIT_TO_CLOSE = num_shares_per_trade * 10
    # -- DO NOT MODIFY THE CODE ABOVE -- #
    # start trading from index 50, where we start to have APO
    last_risk_change_index = 49
    for i in range(50, len(df)):
        # First check if num_shares_per_trade exceeds risk_limit_max_trade_size
        # If so, risk_violated = True
        if num_shares_per_trade > risk_limit_max_trade_size:
            print("Risk Violation: Number of Shares Per Trade Exceeds Risk Limit")
            risk_violated = True
        
        #MIN_PROFIT_TO_CLOSE = num_shares_per_trade *10
        close_price = df["Adj Close"][i]
        stdev_factor = df["stdev_factor"][i]
        apo = df["APO"][i]
        position = df["Positions"][i - 1]
        # At every trading day, we do ONE of the following three things:
        # perform sell trade, perform buy trade or do nothing
        # Rules:
        # If position < 0 when having a "buy" signal, we will unwind all the position
        
        '''if (position < 0) and df['Order'][i] == 1:
            position = 0
            buy_sum_quantity+=abs(position)
            buy_sum_price_quantity+=close_price*abs(position)
        # If position <= 0 when having a "sell" signal, we will short (num_shares_per_trade) shares
        elif (position <= 0) and df['Order'][i] == -1:
            position -= num_shares_per_trade
            sell_sum_quantity+=num_shares_per_trade
            sell_sum_price_quantity= close_price*num_shares_per_trade
        # If position >= 0 when having a "buy" signal, we will long (num_shares_per_trade) shares
        elif (position >= 0) and df['Order'][i] == 1:
            position += num_shares_per_trade
            buy_sum_quantity+=num_shares_per_trade
            buy_sum_price_quantity=close_price*num_shares_per_trade
        # If position > 0 when having a "sell" signal, we will sell all the position
        elif (position > 0) and df['Order'][i] == -1:
            position = 0
            sell_sum_quantity+=position
            sell_sum_price_quantity=close_price*position'''
            
            
        # Perform sell trade at adjusted close price if 
        # (1) risk_violated = False AND
        # (2) The APO > APO_VALUE_FOR_SELL_ENTRY * stdev_factor and the difference between last trade price and curren price is different enough OR
        # (3) We have long position AND (APO is at or above 0 or current position is profitable enough to lock profit)
        #riskcheck = (risk_violated == False)
        #sellapo = apo > APO_VALUE_FOR_SELL_ENTRY*stdev_factor
        #sellprice = abs(close_price - last_sell_price) >(MIN_PRICE_MOVE_FROM_LAST_TRADE * stdev_factor)
        #profit = (unrealized_pnl > (MIN_PROFIT_TO_CLOSE / stdev_factor))
        #sellcond = riskcheck and ((sellapo and sellprice) or position > 0 and profit)
        
        # HINT: "the difference between last trade price and current price is different enough" means "abs(close_price - last_sell_price) > MIN_PRICE_MOVE_FROM_LAST_TRADE * df["stdev_factor"][i]"
        # HINT: "current position is profitable enough to lock profit" means "unrealized_pnl > MIN_PROFIT_TO_CLOSE / stdev_factor"

        # Perform buy trade at adjusted close price if 
        # (1) risk_violated = False AND
        # (2) The APO < APO_VALUE_FOR_BUY_ENTRY * stdev_factor and the difference between last trade price and current price is different enough OR
        # (3) We have short position AND (APO is at or below 0 or current position is profitable enough to lock profit)
        #riskcheck = (risk_violated == False)
        #sellapo = apo < APO_VALUE_FOR_SELL_ENTRY*stdev_factor
        #sellprice = abs(close_price - last_buy_price) >(MIN_PRICE_MOVE_FROM_LAST_TRADE * stdev_factor)
        #profit = (unrealized_pnl > (MIN_PROFIT_TO_CLOSE / stdev_factor))
        #buycond = riskcheck and ((sellapo and sellprice) or position < 0 and profit)
        
        if (not risk_violated and((apo > APO_VALUE_FOR_SELL_ENTRY * stdev_factor and abs(close_price - last_sell_price) > MIN_PRICE_MOVE_FROM_LAST_TRADE * stdev_factor) or(position > 0 and (apo >= 0 or unrealized_pnl > MIN_PROFIT_TO_CLOSE / stdev_factor)))):
            df['Order'][i] = -1
            last_sell_price = close_price
            if position <= 0:  # opening a new entry position
                position -= num_shares_per_trade  # reduce position by the size of this trade
                sell_sum_price_quantity += (close_price * num_shares_per_trade)  # update vwap sell-price
                sell_sum_quantity += num_shares_per_trade
                traded_volume += num_shares_per_trade
            else:  # closing an existing position
                sell_sum_price_quantity += (close_price * abs(position))  # update vwap sell-price
                sell_sum_quantity += abs(position)
                traded_volume += abs(position)
                position = 0  # reduce position by the size of this trade
        # Perform buy trade at adjusted close price if
        # (1) risk_violated = False AND
        # (2) The APO < APO_VALUE_FOR_BUY_ENTRY * stdev_factor and the difference between last trade price and current price is different enough OR
        # (3) We have short position AND (APO is at or below 0 or current position is profitable enough to lock profit)
        elif (not risk_violated and((apo < APO_VALUE_FOR_BUY_ENTRY * stdev_factor and abs(close_price - last_buy_price) > MIN_PRICE_MOVE_FROM_LAST_TRADE * stdev_factor)or
        (position < 0 and (apo <= 0 or unrealized_pnl > MIN_PROFIT_TO_CLOSE / stdev_factor)))):
            df['Order'][i] = 1
            last_buy_price = close_price
            if position >= 0:
                position += num_shares_per_trade
                buy_sum_price_quantity += (close_price * num_shares_per_trade)
                buy_sum_quantity += num_shares_per_trade
                traded_volume += num_shares_per_trade
            else:  # closing an existing position
                buy_sum_price_quantity += (close_price * abs(position))
                buy_sum_quantity += abs(position)
                traded_volume += abs(position)
                position = 0
        else:
            df['Order'][i] = 0
    
        if current_pos == 0:
            if position != 0:
                current_pos = position
                current_pos_start = i
        # check if the holding time of stock exceeds RISK_LIMIT_MAX_POSITION_HOLDING_TIME_DAYS
        # If so, risk_violated = True
        elif current_pos * position <= 0:
            current_pos = position
            position_holding_time = i - current_pos_start
            current_pos_start = i
            # check if the number of position of stock exceeds risk_limit_max_position
            # If so, risk_violated = True
            if position_holding_time > RISK_LIMIT_MAX_POSITION_HOLDING_TIME_DAYS:
                risk_violated = True
            
        # check if the holding time of stock exceeds RISK_LIMIT_MAX_POSITION_HOLDING_TIME_DAYS
        #if position_holding_time > RISK_LIMIT_MAX_POSITION_HOLDING_TIME_DAYS:
        #    risk_violated = True
        # If so, risk_violated = True
        
        # check if the number of position of stock exceeds risk_limit_max_position
        # If so, risk_violated = True
        if (abs(position) > risk_limit_max_position):
            risk_violated = True
        # update open/unrealized & closed/realized positions
        unrealized_pnl = 0

        # Case 1: we have long positions
        # Case 2: we have short positions
        # Case 3: we have no positions
        # update realized_pnl and reset buy_sum_price_quantity, buy_sum_quantity and last_buy_price and their "sell" counterparts
        if position>0:
            unrealized_pnl+= abs(position) * (close_price - buy_sum_price_quantity / buy_sum_quantity)
        elif position<0:
            unrealized_pnl += abs(position) * (sell_sum_price_quantity /sell_sum_quantity - close_price)
        else:#reset
            realized_pnl+= (sell_sum_price_quantity - buy_sum_price_quantity)
            buy_sum_price_quantity = 0
            buy_sum_quantity = 0
            sell_sum_price_quantity = 0
            sell_sum_quantity = 0
            last_buy_price = 0
            last_sell_price = 0
        
        # PnL = realized_pnl + unrealized_pnl
        df["PnL"][i] = realized_pnl + unrealized_pnl
        # Analyze monthly performance and adjust risk up/down with th following rule:
        # (1) If monthly PnL is positive, then increase risk limit for num shares per trade, weekly stop loss, monthly stop loss, max position and trade size
        #    the amount of increase depends on INCREMENT_NUM_SHARES_PER_TRADE, INCREMENT_RISK_LIMIT_WEEKLY_STOP_LOSS, INCREMENT_RISK_LIMIT_MONTHLY_STOP_LOSS, INCREMENT_RISK_LIMIT_MAX_POSITION, INCREMENT_RISK_LIMIT_MAX_TRADE_SIZE
        # (2) If monthly PnL is negative, then decrease risk limit for num shares per trade, weekly stop loss, monthly stop loss, max position and trade size
        #     the amount of decrease depends on INCREMENT_NUM_SHARES_PER_TRADE, INCREMENT_RISK_LIMIT_WEEKLY_STOP_LOSS, INCREMENT_RISK_LIMIT_MONTHLY_STOP
        # (3) Do NOT change the risk limits more than once within 20 trading days (so you should update the last_risk_change_index) 
        monthly_pnls = df['PnL'][i] - df['PnL'][i-20]
        if i - last_risk_change_index > 20:
            if monthly_pnls > 0:
                num_shares_per_trade += INCREMENT_NUM_SHARES_PER_TRADE
                if num_shares_per_trade <= MAX_NUM_SHARES_PER_TRADE:
                    risk_limit_weekly_stop_loss += INCREMENT_RISK_LIMIT_WEEKLY_STOP_LOSS
                    risk_limit_monthly_stop_loss += INCREMENT_RISK_LIMIT_MONTHLY_STOP_LOSS
                    risk_limit_max_position += INCREMENT_RISK_LIMIT_MAX_POSITION
                    risk_limit_max_trade_size += INCREMENT_RISK_LIMIT_MAX_TRADE_SIZE
                else:
                    num_shares_per_trade = MAX_NUM_SHARES_PER_TRADE
            elif monthly_pnls < 0:
                num_shares_per_trade -= INCREMENT_NUM_SHARES_PER_TRADE
                if num_shares_per_trade >= MIN_NUM_SHARES_PER_TRADE:
                    risk_limit_weekly_stop_loss -= INCREMENT_RISK_LIMIT_WEEKLY_STOP_LOSS
                    risk_limit_monthly_stop_loss -= INCREMENT_RISK_LIMIT_MONTHLY_STOP_LOSS
                    risk_limit_max_position -= INCREMENT_RISK_LIMIT_MAX_POSITION
                    risk_limit_max_trade_size -= INCREMENT_RISK_LIMIT_MAX_TRADE_SIZE
                else:
                    num_shares_per_trade = MIN_NUM_SHARES_PER_TRADE
                    
            last_risk_change_index = i
        # Track trade-sizes/positions and risk limits as they evolve over time
        # set value for df["num_shares_history"], df["abs_position_history"], df["max_trade_size_history"] and df["max_position_history"]
        df["PnL"][i] = realized_pnl + unrealized_pnl
        df["num_shares_history"][i] = num_shares_per_trade 
        df["abs_position_history"][i] = abs(position)
        df["max_trade_size_history"][i] = risk_limit_max_trade_size
        df["max_position_history"][i] = risk_limit_max_position

        # Check if weekly loss or monthly loss exceeds risk limits
        # If violated, risk_violated = True
        weekly_loss = df['PnL'][i] - df["PnL"][i-5]

        monthly_loss = df['PnL'][i] - df["PnL"][i-20]
        
        if (weekly_loss < risk_limit_weekly_stop_loss) or (monthly_loss < risk_limit_monthly_stop_loss):
            risk_violated = True

        # update the risk_violated column
        df["risk_violated"][i] = risk_violated
        df['Positions'][i] = position
    return df

def test1(df):
    c = compute_apo(df)
    fptr.write(c["APO"].tail(20).to_string())

def test2(df):
    c = compute_apo(df)
    df = trade_with_risk_controls(c)
    fptr.write(df["Order"][80:105].to_string())

def test3(df):
    c = compute_apo(df)
    df = trade_with_risk_controls(c)
    fptr.write(df["Positions"][80:105].to_string())


def test4(df):
    c = compute_apo(df)
    df = trade_with_risk_controls(c)
    fptr.write(df["PnL"][90:120].to_string())

def test5(df):
    c = compute_apo(df)
    df = trade_with_risk_controls(c)
    fptr.write(df["num_shares_history"][90:120].to_string())
    
def test6(df):
    c = compute_apo(df)
    df = trade_with_risk_controls(c)
    fptr.write(df["abs_position_history"][90:120].to_string())
    
def test7(df):
    c = compute_apo(df)
    df = trade_with_risk_controls(c)
    fptr.write(df["max_trade_size_history"][120:135].to_string())

def test8(df):
    c = compute_apo(df)
    df = trade_with_risk_controls(c)
    fptr.write(df["max_position_history"][90:110].to_string())
    
def test9(df):
    c = compute_apo(df)
    df = trade_with_risk_controls(c)
    fptr.write(df["risk_violated"][95:125].to_string())
    

if __name__ == '__main__':
    fptr = open(os.environ['OUTPUT_PATH'], 'w')
    tmp = input()
    rows_num = int(input().strip())
    
    data = []
    colnames = list(map(str, input().rstrip().split('\t')))
    for i in range(rows_num):
        line = list(map(str, input().split('\t')))
        line[0] = line[0]
        line[1] = float(line[1])
        data.append(line)    

    df = pd.DataFrame(data, columns = colnames)  
    
    if tmp == '1':
        test1(df)
    elif tmp == '2':
        test2(df)
    elif tmp == '3':
        test3(df)
    elif tmp == '4':
        test4(df)
    elif tmp == '5':
        test5(df)
    elif tmp == '6':
        test6(df)
    elif tmp == '7':
        test7(df)
    elif tmp == '8':
        test8(df)
    elif tmp == '9':
        test9(df)
    else:
        raise RuntimeError('invalid input')